In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import folium
from folium.plugins import HeatMap
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import cm as cm

In [2]:
data = pd.read_csv('./Desktop/week2_file.csv', engine = 'python', index_col = 0)

data.head()

,0,1,2,3,4,5,6,7,8,9,...,734,735,736,737,738,739,740,741,742,743
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
areas_ride = data.sum(axis = 1)

zero_areas = len(areas_ride[areas_ride ==0])

print zero_areas #amount of areas with 0 trips

1283


In [4]:
coor = pd.read_csv('./Desktop/regions.csv', sep = ';', engine = 'python')
x = np.union1d(np.unique(coor['east']),np.unique(coor['west']))
y = np.union1d(np.unique(coor['north']),np.unique(coor['south']))

In [5]:
plt.figure(figsize = (250,250))

nyc = Basemap(projection = 'mill', lat_0 =0, lon_0 = 3, llcrnrlat=40.49612,urcrnrlat=40.91553,\
            llcrnrlon=-74.25559,urcrnrlon=-73.70001, resolution = 'f', area_thresh = 0.0001)

nyc.drawcoastlines()
nyc.drawcountries(color='#CCD1D1')
nyc.drawmapboundary(fill_color='aqua')
nyc.drawrivers()

xs, ys = nyc(x[:-1], y[:-1])

cdict = [('white')] + [(cm.jet(i)) for i in xrange(1,256)]
custom_map = LinearSegmentedColormap.from_list('custom_map', cdict, N=256)
plt.register_cmap(cmap=custom_map)


plt.pcolormesh(xs, ys, np.array(data.sum(axis=1)).reshape(50,50), cmap="custom_map")
cbar = plt.colorbar(orientation='horizontal', shrink=0.625, aspect=20, fraction=0.2,pad=0.02)

xx, yy = nyc(-73.985833, 40.747778)
plt.plot(xx, yy, 'ok', markersize=150)
plt.text(xx, yy, 'ESB', fontsize=250)

plt.show()

In [42]:
nyc = folium.Map(location=[40.747778,-73.985833], zoom_start=11, tiles="CartoDB dark_matter")
heat_data = []

for lat in range(len(y)-1):
    for lon in range(len(x)-1):
        rec = folium.Rectangle([[y[lat], x[lon+1]], [y[lat+1], x[lon]]])
        #rec.add_to(nyc)
        if data.mean(axis = 1)[50*lat+lon+1] <5:
            pass
        else:heat_data.append([(y[lat]+y[lat+1])/2., (x[lon+1]+x[lon])/2., data.mean(axis = 1)[50*lat+lon+1]])

HeatMap(heat_data).add_to(nyc)

lib_stat_coor = (40.6892494,-74.0466891)
folium.Marker([40.747778, -73.985833], popup = 'Empire State Building', icon = folium.Icon(color = 'green')).add_to(nyc)
folium.Marker([40.6892499,-74.0463091], popup = 'Statue of Liberty', icon = folium.Icon(color = 'red')).add_to(nyc)
nyc.save("my_map.html")

In [38]:
print len(heat_data)   #areas with more than 5 orders

102
